In [1]:
import Pkg; 

cd(joinpath(@__DIR__, "../../"))
Pkg.activate("Project.toml")

using MorphoMol
using PyCall
using JLD2
using LinearAlgebra
using Rotations
using GLMakie

  Activating project at `~/Doktor/Code/MorphoMol/MorphoMolNotebooks`


In [2]:
py"""
import oineus as oin
import numpy as np
import torch
import diode

def get_interface_diagram(points, n_atoms_per_mol):
    points = np.asarray(points)
    simplices = diode.fill_alpha_shapes(points)
    fil = oin.Filtration_double([oin.Simplex_double(s[0], s[1]) for s in simplices])

    def is_multi(sigma):
        has_a = has_b = False
        for v in sigma.vertices:
            if v < n_atoms_per_mol:
                has_a = True
            else:
                has_b = True
        return has_a and has_b

    fil = fil.subfiltration(is_multi)
    # second argument: True for cohomology, False for homology (incorrect for subfiltration)
    dcmp = oin.Decomposition(fil, True)
    params = oin.ReductionParams()
    params.clearing_opt = False
    dcmp.reduce(params)
    dgm = dcmp.diagram(fil, include_inf_points=False)
    return dgm
"""

function get_interface_diagram(points, n_atoms_per_mol)
    py"get_interface_diagram"(points, n_atoms_per_mol)
end

function get_interface_diagrams(folder)
    files = readdir(folder)
    n = length(files)
    n_atoms_per_mol = Int(length(txt_to_points("$(folder)0.txt")) / 2) # Hardcoded for two sub units
    [get_interface_diagram(txt_to_points("$(folder)$(i-1).txt"), n_atoms_per_mol) for i in 1:n]
end


get_interface_diagrams (generic function with 1 method)

In [3]:
function get_total_persistence(dgms_per_dimension::Vector{Matrix{Float64}}, weight::Float64 = 1.0)
    sum([get_persistence(dgm, weight) for dgm in dgms_per_dimension])
end

function get_persistence(dgm, weight::Float64 = 1.0)
    weight * sum(dgm[:,2] - dgm[:,1])
end

function get_tps(dgms)
    [get_total_persistence(dgm) for dgm in dgms]
end

function get_fsols(folder)
    files = readdir(folder)
    n = length(files)
    [parse(Float64, split(readlines("$(folder)$(i-1).txt")[2], " ")[6]) for i in 1:n]
end

function get_thetas(folder)
    files = readdir(folder)
    n = length(files)
    [parse(Float64, split(readlines("$(folder)$(i-1).txt")[2], " ")[7]) for i in 1:n]
end

function txt_to_points(input_file)
    [[parse(Float64, s) for s in split(p," ")] for p in readlines(input_file)[3:end]]
end

txt_to_points (generic function with 1 method)

In [4]:
function get_configurations_from_folder(folder)
    files = readdir(folder)
    realizations = [Vector{Vector{Vector{Float64}}}([]) for i in 1:length(files)]
    for file in readdir(folder)
        centers = txt_to_points("$(folder)$(file)")
        n_atoms_per_mol = Int(length(centers) / 2)
        c1 = centers[1:n_atoms_per_mol]
        c2 = centers[n_atoms_per_mol+1:end]
        realizations[parse(Int, split(file, ".")[1]) + 1] = [c1, c2]
    end
    realizations
end

get_configurations_from_folder (generic function with 1 method)

In [5]:
function folder_of_txts_to_jld2_with_persistence_computation(folder)
    mol_type = split(folder, "/")[end-2]
    idx = split(folder, "/")[end-1]
    py_dgms = get_interface_diagrams(folder)
    dgms = [[dgm[1], dgm[2], dgm[3], dgm[4]] for dgm in py_dgms]
    tps = get_tps(dgms)
    Es = get_fsols(folder)
    thetas = get_thetas(folder)
    realizations = get_configurations_from_folder(folder)
    @save "../../Data/measure_paths/jld2s/$(mol_type)/$(idx).jld2" dgms tps Es thetas realizations
end

folder_of_txts_to_jld2_with_persistence_computation (generic function with 1 method)

In [35]:
# folder = "../../Data/measure_paths/txts/6r7m/"
# for subdir in readdir(folder)
#     if subdir != ".DS_Store"
#         folder_of_txts_to_jld2_with_persistence_computation("$(folder)$(subdir)/")
#     end
# end

# folder = "../../Data/measure_paths/txts/6r7m/1/"
# dgms = get_interface_diagrams(folder)
# xs = [i for i in 1:length(dgms)]
# dgms_selection = [[dgm[2]; dgm[3]] for dgm in dgms]
# tps = get_tps(dgms_selection)
# Es = get_fsols(folder)
# thetas = get_thetas(folder);

In [6]:
@load "../../Data/measure_paths/jld2s/6r7m/1.jld2" dgms tps Es thetas realizations
xs = [i for i in 1:length(dgms)];

In [7]:
two_ps = [get_persistence(dgm[2], -0.1) for dgm in dgms]
three_ps = [get_persistence(dgm[3], -0.1) for dgm in dgms]
tps = [get_total_persistence(dgm, -0.1) for dgm in dgms];

In [8]:
f = Figure(fontsize = 7)
pd_ax_a = Axis(f[1, 1], title = "Persistence codim = 2", xticks = 0:25:100, yticks = 0:25:100)
pd_ax_b = Axis(f[1, 2], title = "Persistence codim = 3", xticks = 0:25:100, yticks = 0:25:100)
pd_hm_ax_a = Axis(f[2, 1], title = "Persistence codim = 2")
pd_hm_ax_b = Axis(f[2, 2], title = "Persistence codim = 3")

conf_ax = Axis3(f[1, 3])
tp_ax = Axis(f[3, 1], title = "Total Persistence")
theta_ax = Axis(f[3, 2], title = L"\Theta")
E_ax = Axis(f[3, 3], title = L"F_{sol}")
two_p_ax = Axis(f[4, 1], title = "codim 2 persistence")
three_p_ax = Axis(f[4, 2], title = "codim 3 persistence")

sl_i = Slider(f[5, 1:3], range = 1:length(dgms), startvalue = 1)

x = sl_i.value

tp_mark = @lift(Point2f($x, $@lift(tps[$x])))
theta_mark = @lift(Point2f($x, $@lift(thetas[$x])))
E_mark = @lift(Point2f($x, $@lift(Es[$x])))
two_p_mark = @lift(Point2f($x, $@lift(two_ps[$x])))
three_p_mark = @lift(Point2f($x, $@lift(three_ps[$x])))

dgm_points_2 = @lift([Point2f(dgms[$x][2][i,1], dgms[$x][2][i,2]) for i in 1:size(dgms[$x][2])[1]])
dgm_points_3 = @lift([Point2f(dgms[$x][3][i,1], dgms[$x][3][i,2]) for i in 1:size(dgms[$x][3])[1]])

c1_points = @lift([Point3f(realizations[$x][1][i][1], realizations[$x][1][i][2], realizations[$x][1][i][3]) for i in 1:length(realizations[$x][1])])
c2_points = @lift([Point3f(realizations[$x][2][i][1], realizations[$x][2][i][2], realizations[$x][2][i][3]) for i in 1:length(realizations[$x][2])])

ms = 5
scatter!(pd_ax_a, dgm_points_2, color = :orange, markersize = ms)
lines!(pd_ax_a, [0, 100], [0, 100], color=:black)
scatter!(pd_ax_b, dgm_points_3, color = :green, markersize = ms)
lines!(pd_ax_b, [0, 100], [0, 100], color=:black)

#contourf!(pd_hm_ax_a, dgm_points_2, levels = 10)
#contourf!(pd_hm_ax_b, dgm_points_3, levels = 10)

scatter!(tp_ax, xs, tps, color=:red, markersize = ms)
scatter!(tp_ax, tp_mark, color = :black, markersize = 10)

conf_ms = 10
scatter!(conf_ax, c1_points, markersize = conf_ms)
scatter!(conf_ax, c2_points, markersize = conf_ms)

scatter!(theta_ax, xs, thetas, color=:magenta, markersize = ms)
scatter!(theta_ax, theta_mark, color = :black, markersize = 10)

scatter!(E_ax, xs, Es, color=:orange, markersize = ms)
scatter!(E_ax, E_mark, color = :black, markersize = 10)

scatter!(two_p_ax, xs, two_ps, color=:orange, markersize = ms)
scatter!(two_p_ax, two_p_mark, color = :black, markersize = 10)

scatter!(three_p_ax, xs, three_ps, color=:green, markersize = ms)
scatter!(three_p_ax, three_p_mark, color = :black, markersize = 10)

display(f)


GLMakie.Screen(...)